In [1]:
import sys
import pandas as pd
import dateutil
import numpy as np
sys.path.append('../')
from utilities.era5_down import era5_down
import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
#cs_file = '../data/SMB_bellin_season.csv'
cs_file = '../data/SMB_four_season.csv'
dfmb = pd.read_csv(cs_file,
   delimiter='\t', parse_dates=['Date', 'Date1'], na_values='NAN')
labels = dfmb.drop_duplicates(subset=['Stake'])

In [5]:
t2m = 't2m'
rh2 = 'rh'
ff  = 'u2'
Prec = 'tp'
Snowfall = 'sf'
SWD = 'SWin'
LWD = 'LWin'
Pres = 'press'
fcc  = 'tcc'
msl  = 'msl'

In [6]:
files = sorted(glob.glob('../data/ERA_59_20_day/*.nc'))
len(files)

62

In [7]:
data_pro = []
for st in labels['Stake']:
    stake  = dfmb.loc[lambda df: df['Stake'] == st, :].copy().reset_index()
    df_day = era5_down(files, stake['Longitude'][0],
                       stake['Latitude'][0], 
                       stake['Elevation'][0])
    df_day['t2m_an'] = (df_day[t2m] - df_day[t2m].mean())/df_day[t2m].std()
    subset_f0 = []
    for i in range(len(stake)):
        subset = df_day.loc[str(stake.Date1[i]): str(stake.Date[i])].copy()
        subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0)
        subset['stake'] = stake['Stake'][0]
        subset_f = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                         SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                         Snowfall:nansumwrapper, msl:np.mean,
                                         Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                         'PDD':nansumwrapper})
        subset_f.index = ([stake.Date[i]])
        subset_f0.append(subset_f)
    
    data_pro.append(pd.concat([stake, pd.concat(subset_f0).reset_index()], axis=1))

In [8]:
data_inp_four = pd.concat(data_pro)
data_inp_four.drop(['index', 'Date1', 'index'], axis=1, inplace=True)

In [9]:
data_inp_four.columns

Index(['Stake', 'Date', 'Latitude', 'Longitude', 'Elevation', 'Days', 'SMB',
       't2m', 'rh', 'u2', 'SWin', 'LWin', 'tp', 'sf', 'msl', 'press', 'tcc',
       't2m_an', 'PDD'],
      dtype='object')

In [10]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,rh,u2,SWin,LWin,tp,sf,msl,press,tcc,t2m_an,PDD
0,PG19,2010-05-01,-62.237672,-58.633624,96.8,0,2.12,-10.172760,67.462929,4.397378,33.654079,200.144638,0.000029,0.000028,986.502197,974.859680,0.783153,-1.790036,0.000000
1,PG19,2010-11-22,-62.237672,-58.633624,96.8,205,2.12,-2.845091,85.940948,11.540661,58.478569,269.106079,0.579979,0.497697,989.917358,978.396240,0.863928,0.037833,3.770416
2,PG19,2011-04-06,-62.237672,-58.633624,96.8,135,-1.20,-0.137477,86.900452,9.719824,175.098328,278.426331,0.348022,0.197304,988.769226,977.310120,0.819164,0.713240,69.808533
3,PG19,2011-10-15,-62.237672,-58.633624,96.8,192,0.83,-6.329857,84.209000,9.314919,35.074932,254.948547,0.459260,0.402851,994.918579,983.274597,0.873794,-0.831433,2.194000
4,PG19,2012-04-21,-62.237672,-58.633624,96.8,189,-3.45,-1.037745,86.991310,10.708667,162.158783,273.473175,0.476021,0.317661,987.722412,976.258667,0.818269,0.488671,53.203003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,PG05,2014-04-30,-62.227069,-58.598800,247.9,155,-0.03,-2.333732,85.016083,9.040595,170.536514,269.368866,0.328569,0.260867,989.707458,960.822876,0.811101,0.509413,0.842102
9,PG05,2014-11-30,-62.227069,-58.598800,247.9,214,1.27,-5.252601,86.123360,10.295300,66.620850,265.289337,0.566755,0.533022,991.736084,962.728943,0.883004,-0.200674,0.000000
10,PG05,2015-04-13,-62.227069,-58.598800,247.9,134,-0.83,-1.515892,88.116119,9.455073,170.939133,275.884399,0.270530,0.173233,987.971130,959.158569,0.796739,0.708372,10.097656
11,PG05,2015-11-19,-62.227069,-58.598800,247.9,220,1.08,-6.548934,84.148323,10.761319,52.485832,258.022095,0.612647,0.576701,987.028137,958.126221,0.885483,-0.516039,0.484192


In [11]:
# reset SMB, tp, snowfall and PDD to 0 on 0 days observation
#data_inp_four[['SMB',Prec, Snowfall, 'PDD' ]] = data_inp_four[['SMB',Prec, Snowfall,
#                                                               'PDD' ]].where(data_inp_four['Days'] <= 100,
#                                                                              np.nan)
data_inp_four[['SMB',Prec, Snowfall, 'PDD' ]] = data_inp_four[['SMB',Prec, Snowfall,
                                                               'PDD' ]].where(data_inp_four['Days'] != 0,
                                                                              np.nan)

#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['SMB'] < 0, 0)
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 260, 0) # four
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 200, 0) # bell


data_inp_four.dropna(inplace=True)

In [12]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,rh,u2,SWin,LWin,tp,sf,msl,press,tcc,t2m_an,PDD
1,PG19,2010-11-22,-62.237672,-58.633624,96.8,205,2.12,-2.845091,85.940948,11.540661,58.478569,269.106079,0.579979,0.497697,989.917358,978.396240,0.863928,0.037833,3.770416
2,PG19,2011-04-06,-62.237672,-58.633624,96.8,135,-1.20,-0.137477,86.900452,9.719824,175.098328,278.426331,0.348022,0.197304,988.769226,977.310120,0.819164,0.713240,69.808533
3,PG19,2011-10-15,-62.237672,-58.633624,96.8,192,0.83,-6.329857,84.209000,9.314919,35.074932,254.948547,0.459260,0.402851,994.918579,983.274597,0.873794,-0.831433,2.194000
4,PG19,2012-04-21,-62.237672,-58.633624,96.8,189,-3.45,-1.037745,86.991310,10.708667,162.158783,273.473175,0.476021,0.317661,987.722412,976.258667,0.818269,0.488671,53.203003
5,PG19,2012-11-24,-62.237672,-58.633624,96.8,217,2.50,-4.113695,85.382713,10.679875,59.147972,264.708008,0.638591,0.594895,989.141907,977.605469,0.882287,-0.278617,1.946411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,PG05,2014-04-30,-62.227069,-58.598800,247.9,155,-0.03,-2.333732,85.016083,9.040595,170.536514,269.368866,0.328569,0.260867,989.707458,960.822876,0.811101,0.509413,0.842102
9,PG05,2014-11-30,-62.227069,-58.598800,247.9,214,1.27,-5.252601,86.123360,10.295300,66.620850,265.289337,0.566755,0.533022,991.736084,962.728943,0.883004,-0.200674,0.000000
10,PG05,2015-04-13,-62.227069,-58.598800,247.9,134,-0.83,-1.515892,88.116119,9.455073,170.939133,275.884399,0.270530,0.173233,987.971130,959.158569,0.796739,0.708372,10.097656
11,PG05,2015-11-19,-62.227069,-58.598800,247.9,220,1.08,-6.548934,84.148323,10.761319,52.485832,258.022095,0.612647,0.576701,987.028137,958.126221,0.885483,-0.516039,0.484192


In [13]:
#data_inp_four.to_csv('../data/SMB_input_belli_sea_ERA5.csv', index=False, sep='\t')
data_inp_four.to_csv('../data/SMB_input_four_sea_ERA5.csv', index=False, sep='\t')